In [8]:
import numpy as np
import pandas as pd
import os
import os.path as op
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from julearn import run_cross_validation
from julearn.utils import configure_logging
from sklearn.model_selection import train_test_split, StratifiedKFold
# models
from skrvm import RVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso, ElasticNet
import sklearn.gaussian_process as gp
import sys
sys.path.append("C:/Users/doeringe/Documents/BrainAge/PET_MRI_age/lib")
from create_splits import stratified_splits

df = pd.read_csv('test_train_stratified.csv',index_col=0)

df_train = df[df['train'] == "T"]

X = df_train.values[:,1:-3]
y_pseudo = df_train['AGE_bins']
y = df_train['AGE'].values

print(np.bincount(y_pseudo))

# Define all models
rvr = RVR()
#pls = PLSRegression() 
kernel_ridge = KernelRidge() 
mlp = MLPRegressor()
lasso = Lasso(alpha=1)
elasticnet = ElasticNet(alpha=0.5)

model_list = ['ridge', 'rf', rvr, kernel_ridge, 'gauss', lasso, elasticnet]
model_names = ['ridge', 'rf', 'rvr', 'kernel_ridge', 'gauss', 'lasso', 'elasticnet']

model_para_list = [{'ridge__alpha': [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 'cv': 5},
                   {'rf__n_estimators': 500, 'rf__criterion': 'mse', 'cv': 5},
                   {'rvr__kernel': 'linear', 'cv': 5},
                   {'kernelridge__alpha': [0.0, 0.001, 0.01, 0.1, 0.5, 1.0, 10.0, 100.0, 1000.0], 'kernelridge__kernel': 'polynomial',
                    'kernelridge__degree': [1, 2], 'cv': 5},
                   {'gauss__kernel': gp.kernels.ConstantKernel(1.0, (1e-1, 1e3)) * gp.kernels.RBF(10.0,
                  (1e-3, 1e3)),
                    'gauss__n_restarts_optimizer': 10, 'gauss__alpha': 0.1, 'gauss__normalize_y': True,
                    'cv': 5}]
                   
rand_seed = 42
num_bins = 5
                   
for i in range(0, len(model_names)):
        print(i, model_names[i])

        if model_names[i] == 'mlp' and model_names[i] == 'lasso_new':
            preprocess_X = 'zscore'
        else:
            preprocess_X = None

        if model_names[i] != 'elasticnet' and model_names[i] != 'lasso':
            cv = StratifiedKFold(n_splits=5).split(X, y_pseudo)
            scores, model = run_cross_validation(X=X, y=y, data=df_train, preprocess_X=preprocess_X,
                                                problem_type='regression', model=model_list[i], cv=cv,
                                                return_estimator='final', model_params=model_para_list[i], 
                                                seed=rand_seed,
                                                scoring=['neg_mean_absolute_error', 'neg_mean_squared_error','r2'])
        else:
                cv = StratifiedKFold(n_splits=5).split(X, y_pseudo)
                scores, model = run_cross_validation(X=X, y=y, data=df_train, preprocess_X=preprocess_X,
                                                     problem_type='regression', model=model_list[i], cv=cv,
                                                     return_estimator='final', seed=rand_seed, scoring=
                                                     ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'])


"""# creates dictionary of test indices for different repeats
cv_folds = stratified_splits(bins_on=y_pseudo, num_bins=num_bins,
                             data=df_train, num_splits=5,
                             shuffle=True, random_state=42)
cv = StratifiedKFold(n_splits=5).split(X, y_pseudo)

scores,model = run_cross_validation(X=X, y=y,
                              preprocess_X='scaler_robust',
                              problem_type='regression',
                              model='gradientboost', cv=cv,
                              return_estimator='final',
                              seed=rand_seed)"""


X must be a string or list of strings


[57 54 56 58 53 54 61 51 53 55]
0 ridge


ValueError: X must be a string or list of strings

In [46]:
print(scores)

   fit_time  score_time  test_score  repeat  fold
0  2.860007    0.031264    0.999835       0     0
1  2.840779    0.037752    0.999670       0     1
2  2.753733    0.053517    0.999831       0     2
3  2.944075    0.031279    0.999744       0     3
4  3.146703    0.031237    0.999786       0     4


In [47]:
df_test = df[df['train'] == "F"]

X_test = df_test.values[:,1:-3]
y_test_pseudo = df_test['AGE_bins']
y = df_test['AGE'].values

print(np.bincount(y_test_pseudo))

[15 13 14 15 13 13 15 13 14 14]


In [59]:
print(model.predict(pd.DataFrame(X_test)))

KeyError: "None of [Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',\n       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',\n       ...\n       'feature_131', 'feature_132', 'feature_133', 'feature_134',\n       'feature_135', 'feature_136', 'feature_137', 'feature_138',\n       'feature_139', 'feature_140'],\n      dtype='object', length=141)] are in the [columns]"